**這個檔案必須要在Google Colab上執行，否則一些檔案路徑要改。還有一些Linux的指令，如果是在Windows執行要更改。Windows安裝Tensorflow建議使用Anaconda安裝，其中在Search完後可這樣裝conda install  package_name = version = build_string裡面的build_string前三位是python版本要注意。Anaconda安裝GPU版本會自己解決環境問題，幫你自動裝好Cuda Tookit、Cudnn。然後記得要windows long path support enabled。**

**用Windows安裝pycocotools要先下載visual studio build tools**

In [ ]:
"""
Tensorflow官方網站:
https://www.tensorflow.org/

原始教學程式碼來源:
https://github.com/tensorflow/models/blob/master/research/object_detection/colab_tutorials/object_detection_tutorial.ipynb
https://blog.gtwang.org/programming/tensorflow-object-detection-api-tutorial/
https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/auto_examples/plot_object_detection_saved_model.html#sphx-glr-auto-examples-plot-object-detection-saved-model-py
https://ithelp.ithome.com.tw/m/articles/10238231

models source:
https://cocodataset.org/#home
https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md

這個模組能分辨的物體:
https://cocodataset.org/#explore

"""

In [ ]:
import platform
print(platform.system(),"\n",platform.uname())

In [ ]:
!pip install tensorflow
!pip install matplotlib
!pip install tf_slim
!pip install pycocotools
!pip install pillow
!pip install requests
!pip install lxml

In [ ]:
!git clone https://github.com/tensorflow/models.git

In [ ]:
cd models/research

In [ ]:
!apt install -y protobuf-compiler

In [ ]:
!protoc object_detection/protos/*.proto --python_out=.

In [ ]:
!export PYTHONPATH=$PYTHONPATH:`pwd`:`pwd`/slim

In [ ]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
import requests
import shutil
from lxml import etree
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display
from tensorflow.keras import backend
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
# patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [ ]:
def load_model(model_name):
  base_url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/'
  model_file = model_name + '.tar.gz'
  model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)

  model_dir = pathlib.Path(model_dir)/"saved_model"

  model = tf.saved_model.load(str(model_dir))

  return model

In [ ]:
# List of the strings that is used to add correct label for each box.
PATH_TO_LABELS = '/content/models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

In [ ]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('/content/models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

In [ ]:
#model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
model_name="efficientdet_d6_coco17_tpu-32"
detection_model = load_model(model_name)

In [ ]:
def run_inference_for_single_image(model, image):
  image = np.asarray(image)
  # The input needs to be a tensor, convert it using `tf.convert_to_tensor`.
  input_tensor = tf.convert_to_tensor(image)
  # The model expects a batch of images, so add an axis with `tf.newaxis`.
  input_tensor = input_tensor[tf.newaxis,...]

  # Run inference
  model_fn = model.signatures['serving_default']
  output_dict = model_fn(input_tensor)

  # All outputs are batches tensors.
  # Convert to numpy arrays, and take index [0] to remove the batch dimension.
  # We're only interested in the first num_detections.
  num_detections = int(output_dict.pop('num_detections'))
  output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
  output_dict['num_detections'] = num_detections

  # detection_classes should be ints.
  output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
   
  # Handle models with masks:
  if 'detection_masks' in output_dict:
    # Reframe the the bbox mask to the image size.
    detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
              output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
    detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,
                                       tf.uint8)
    output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    
  return output_dict

In [ ]:
print(detection_model.signatures['serving_default'].inputs)

In [ ]:
detection_model.signatures['serving_default'].output_dtypes

In [ ]:
detection_model.signatures['serving_default'].output_shapes

In [ ]:
def show_inference(model, image_path):
  # the array based representation of the image will be used later in order to prepare the
  # result image with boxes and labels on it.
  image_np = np.array(Image.open(image_path))
  # Actual detection.
  output_dict = run_inference_for_single_image(model, image_np)
  # Visualization of the results of a detection.
  vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=8)

  display(Image.fromarray(image_np))

In [ ]:
def get_correct_url(base_url,part_url):
  if(part_url.split('/')[0].find('http')==-1):
    if(part_url.split('/')[0]==''):
      if(part_url.split('/')[0]==''):
        if(part_url.split('/')[1]==''):
          if(part_url.split('/')[2]==base_url.split('/')[2]):
            return "http:"+part_url
          else:
            pass  
        else:
          if(part_url.split('/')[1]==base_url.split('/')[2]):
            return "http:/"+part_url
          else:
            return "http://"+base_url.split("/")[2]+part_url 
      else:  
        return "http://"+base_url.split("/")[2]+"/"+part_url 
    else:
      return "http://"+base_url.split("/")[2]+"/"+part_url       

  else:
    return part_url

In [ ]:
my_imgs_now=[]

def png_to_jpg(mypath):
  im = Image.open("/content/"+mypath)
  rgb_im = im.convert('RGB')
  rgb_im.save("/content/"+mypath+".jpg")

def my_save_img():
  global my_imgs_now
  def my_save_url_img(my_img_url): 
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
    r = requests.get(my_img_url,headers=headers, stream=True)
    if r.status_code == 200:
        with open(str(("/content/"+my_img_url.split('/')[-1].split('?')[0])), 'wb') as f:
            r.raw.decode_content = True
            shutil.copyfileobj(r.raw, f)

  url_01=input("請輸入圖片網址：")
  url_01_type=url_01.split('/')[-1].split('.')[-1].split('?')[0]
  my_imgs_now=[]
  
  if(url_01_type=='jpg'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])
  
  elif(url_01_type=='png'):
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0]+".jpg")
    png_to_jpg(url_01.split('/')[-1].split('?')[0])

  else:
    try:
      #response.status_code,response.content,response.text
      headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/67.0.3396.99 Safari/537.36"}
      my_response = requests.get(url_01,headers=headers)
      my_content = my_response.content.decode()
      my_html = etree.HTML(my_content)
      my_imgs = my_html.xpath('.//img/@src')
      print(my_imgs,"\n")
      for i in my_imgs:
        try:
          if(i.split('/')[-1].split('.')[-1].split('?')[0]=='png'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0]+".jpg")
            png_to_jpg(i.split('/')[-1].split('?')[0])
          elif(i.split('/')[-1].split('.')[-1].split('?')[0]=='jpg'):
            #change this line
            my_save_url_img(get_correct_url(url_01,i))
            my_imgs_now.append(i.split('/')[-1].split('?')[0])    
        except:
          pass
    except:
      pass  


  if(len(my_imgs_now)==0):
    print('圖片網址設定失敗，使用預設圖片')
    url_01="https://64.media.tumblr.com/464a8a1770ac6b2c09c80232911673b7/b6713c87a0ba23de-b6/s1280x1920/0306a7f325e1c10e768a26ac2e1ea805ad3211d5.jpg"
    my_save_url_img(url_01)
    my_imgs_now.append(url_01.split('/')[-1].split('?')[0])  
  else:  
    print('圖片網址設定成功')

In [ ]:
print(get_correct_url('http://pixiv.navirank.com/tag/%E6%BD%A4%E7%BE%BD%E3%82%8B%E3%81%97%E3%81%82/,part_url','img/896/89657023.jpg'))

In [ ]:
my_save_img()

In [ ]:
my_imgs_now

In [ ]:
len(my_imgs_now)

In [ ]:
#for image_path in TEST_IMAGE_PATHS:
#  show_inference(detection_model, image_path)

backend.clear_session()

for i in my_imgs_now:
  show_inference(detection_model,("/content/"+str(i)))